#### ParamLogger
ParamLogger is a library that helps you log inputs and outputs of any method, just by providing a decorator over that method.

Here we demonstarte with example all the utility methods and decorator that library provides.

In [ ]:
from paramlogger import ParamLogger

log_dir = "" # Path to directory where you want to store all log files
paramlog = ParamLogger(base_path=log_dir)

#### 1) log_io_params_for_method decorator

Executes the method. After executing, logs the inputs and outputs of that method to jsonl file. Name of jsonl file would be the method name.

In [ ]:
@paramlog.log_io_params_for_method
def compute_addition(operand1, operand2):
    return operand1 + operand2

for i in range(10):
    compute_addition(i,10*i)

#### 2) log_io_params decorator
Executes the method. After executing, logs the inputs and outputs of that method to jsonl file named "io_logs.jsonl". Here, inputs/outputs of different methods gets saved in the same file.

In [ ]:
@paramlog.log_io_params
def compute_addition(operand1, operand2):
    return operand1 + operand2

@paramlog.log_io_params
def compute_subtraction(operand1, operand2):
    return operand2 - operand1

@paramlog.log_io_params
def compute_multiplication(operand1, operand2):
    return operand2 * operand1

for i in range(1, 10):
    result1 = compute_addition(i,10*i)
    result2 = compute_subtraction(result1, i)
    result3 = compute_multiplication(result2, i)

#### 3) append_to_chained_log decorator
In `log_io_params` and `log_io_params_for_method` decorators, saving the inputs/optputs to file happens after every method call. This can increase the latency of  your program due to frequent file operations. This can be avoided via `append_to_chained_log`. Here, everything to be logged gets saved in main-memory in the form of a python-list. It'll get saved to file only when users asks to do so by calling `dump_chained_log_to_file` method.

In [ ]:
@paramlog.append_to_chained_log
def compute_addition(operand1, operand2):
    return operand1 + operand2


for i in range(1, 100):
    # inputs and outputs of this method call are
    # present in main-memory but not saved to file (disk).
    result1 = compute_addition(i,10*i)


# Here we ask to save all the logs collected for compute_addition()
# method call to file, with only 1 file access operation.
# It'll get saved to file with name "all_compute_addition_logs.jsonl"
paramlog.dump_chained_log_to_file(file_name="all_compute_addition_logs")

#### 4) clear_chained_log() method
Deletes the logs collected so far (before saving to file). This can work only with logs collected via `append_to_chained_log` decorator.

In [ ]:
@paramlog.append_to_chained_log
def compute_addition(operand1, operand2):
    return operand1 + operand2


for i in range(1, 100):
    # inputs and outputs of this method call are
    # present in main-memory but not saved to file (disk).
    result1 = compute_addition(i,10*i)
    if i==50:
        # By this all logs collected for iteration 1 to 50, would be deleted and
        # will not be present in the log file.
        paramlog.clear_chained_log()


# Here we ask to save all the logs collected for compute_addition()
# method call to file, with only 1 file access operation.
# It'll get saved to file with name "all_compute_addition_logs.jsonl"
paramlog.dump_chained_log_to_file(file_name="part_compute_addition_logs")

#### 5) run_over_logs decorator
Runs the method tagged by run_over_logs decorator over each entry in jsonl file that was created by any of the above methods.
And save the result in a new `.jsonl` file, keeping the mapping.

In [ ]:
@paramlog.log_io_params_for_method
def prediction(x, gt):
    # NOTE: we are passing gt (ground truth) as input, but its not getting
    # used in method. We do this so that ground truth gets logged.
    slope=10
    bias=3
    return slope*x + bias

ground_truth = [i*0.1 for i in range(10)]
for x, gt in zip(range(10), ground_truth):
    y = prediction(x, gt)

# After running above code, inputs and outputs of prediction() method would have been logged in prediction.jsonl file


"""
We are using run_over_logs decorator to run absolute_accuracy() method over each record in prediction.jsonl file.
`id`, `inputs`, `outputs` fields in prediction.jsonl can be accessed via `dummy_id`, `dummy_input`, `dummy_output` parameters respectively.
These fields have to be initialized to None, when defining absolute_difference() method.
dummy_input would always be dictionary, so that you can send any number of parameters of any datatype.
Data type of `dummy_output` would be same as datatype of output of prediction() method above.
`file_path` parameter takes path to prediction.jsonl file.
"""
@paramlog.run_over_logs
def absolute_difference(file_path, dummy_id=None, dummy_input=None, dummy_output=None, dummy_meta=None, **kwargs):
    # This method computes absolute difference between ground truth and prediction.
    gt = float(dummy_input["gt"])
    y = dummy_output
    return abs(y-gt)


absolute_accuracy(file_path="prediction.jsonl")